
# Setup


In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import os
import json

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from datetime import date
from pvlib.iotools import read_epw

from pykrige.rk import RegressionKriging
from pykrige.ok import OrdinaryKriging
import pickle

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#dir = 'C:\\GitHub\\microclimate-dl-predict\\data\\'
dir = "C:\\Users\\pkastner\\Documents\\GitHub\\microclimate-dl-predict\\data\\"


# Load, verify and process input files

In [ ]:
measurenames={"RH0719":'RH',"Tem0719":'Temperature'}
measureunit={"Tem0719":'°C',"RH0719":'%'}
measure = "RH0719"
#measure = "Tem0719"
testmeasure = "RH0819"
#testmeasure = "Tem0819"
df = pd.read_csv(dir + measure +'.csv')
testdf = pd.read_csv(dir + testmeasure +'.csv')


# df = df.drop(df.index[2])

amy,meta=read_epw(dir+'SGP_Singapore.486980_IWEC.epw')
changi,meta=read_epw(dir+'SGP_SINGAPORE-CHANGI-AP_486980S_19.epw')
if measure == "Tem0719":
    mykey='temp_air'
else:
    mykey='relative_humidity'
changidata=changi.loc[(changi['year'].astype('str')+changi['month'].astype('str')=='20197'),[mykey]]
amydata=amy.loc[(amy['month'].astype('str')=='7'),[mykey]]

In [ ]:
windspeeddf = pd.read_csv(dir + 'ws0719.csv')
solardf = pd.read_csv(dir + 'sr0719.csv')

In [ ]:
griddf = pd.read_csv(dir + '1m_GridPoints_distTo_and_zones_3414.csv').fillna(0)
wsdf = pd.read_csv(dir + 'WSPoints_TreesAndStreets_3414.csv').fillna(0)
t = pd.concat([griddf, pd.get_dummies(griddf.HorticultureZone)], axis=1)
t = t.replace("Temparea", 1)
t = t.replace("Vegetated Ridge", 1)
# t = t.replace("Valley", 1)
# t = t.rename(columns={"Area": "VegetatedRidge"})
### drop features that share high correlation with other features (not useful for prediction)
griddf = t.drop(['id','HorticultureZone'], axis=1)
# griddf = t.drop(['id','HorticultureZone','gridarea','buildingfootprintarea','roadarea','patharea','walkwayarea','courttrackarea','carparkarea'], axis=1)
### set points within VegetatedRidge to have distToTree 0.0 (trees within the area are not doccumented but the area is densely forrested)
### helps a bit to make distToTree useful but numtrees is still inaccurate for this area
griddf['distToTree'].loc[griddf['VegetatedRidge'] == 1] = 0.0

t = pd.concat([wsdf, pd.get_dummies(wsdf.HorticultureZone)], axis=1)
t = t.replace("Temparea", 1)
t = t.replace("Vegetated Ridge", 1)
t = t.replace("Valley", 1)
# t = t.rename(columns={"Area": "VegetatedRidge"})
t = t.sort_values(by=['ID'])
### drop features that share high correlation with other features (not useful for prediction)
wsdf = t.drop(['Lat','Long','HorticultureZone','Location','Type','Description','gridarea','buildingfootprintarea','roadarea','patharea','walkwayarea','courttrackarea','carparkarea'], axis=1)
wsdf = wsdf.reset_index(drop=True)

plotdf = testdf.transpose().iloc[2:16,:]
plotdf['X']=list(wsdf['X'])
plotdf['Y']=list(wsdf['Y'])

display(wsdf)
display(griddf.head())

C:\Users\pkastner\AppData\Local\Temp\ipykernel_65304\1863566996.py:1: DtypeWarning: Columns (10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  griddf = pd.read_csv(dir + '1m_GridPoints_distTo_and_zones_3414.csv').fillna(0)
C:\Users\pkastner\AppData\Local\Temp\ipykernel_65304\1863566996.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  t = t.replace("Vegetated Ridge", 1)
C:\Users\pkastner\AppData\Local\Temp\ipykernel_65304\1863566996.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update t

,X,Y,ID,distToBuilding,percentbuilt,terrain,Area,Temparea,Valley,distToTree,...,percentroad,percentpath,percentwalkway,percentcourttrack,percentcarpark,numtrees,Zone 1,Zone 2,Zone 3,Zone 4
0,21310.45279,31360.82993,2,1.57015,17.48721,25.03599,0,1,1,0.00008,...,0.00000,14.85820,0.00000,0.000,0.00000,0,True,False,False,False
1,21042.23831,31242.52129,3,0.68151,37.08572,47.24897,0,0,1,0.00043,...,32.90080,0.00000,0.00000,0.000,0.00000,0,True,False,False,False
2,21251.46471,31202.70922,4,5.49422,45.78975,42.84434,0,0,1,0.00005,...,0.00000,32.10427,0.00000,0.000,0.00000,0,True,False,False,False
3,21210.28151,30976.03165,6,2.84506,39.82353,40.05906,0,0,0,0.00008,...,28.49887,26.57608,6.61014,0.000,0.00000,0,False,True,False,False
4,20933.16250,30824.55084,7,9.90256,0.00000,18.97783,0,0,0,0.00002,...,0.00000,0.00000,0.00000,0.000,0.00000,1,False,False,False,True
5,21188.01955,30822.33307,8,0.00000,55.52813,47.79486,0,0,1,0.00007,...,0.00000,0.00000,0.00000,0.000,0.00000,1,False,False,False,True
6,21254.80024,31068.91350,11,0.00000,9.38553,47.58942,0,0,1,0.00005,...,2.45925,1.77000,15.15244,0.000,22.41425,3,True,False,False,False
7,21908.06866,30561.35985,12,10.52352,15.75304,33.60504,1,0,0,0.00015,...,0.00000,0.00000,0.00000,0.000,0.00000,0,False,True,False,False
8,22150.68034,30388.85790,13,19.30173,0.00000,35.78543,0,1,0,0.00019,...,73.29213,0.00000,1.47421,0.000,0.00000,0,False,True,False,False
9,22196.31786,30779.18631,16,21.65002,0.00000,41.24351,0,0,0,0.00007,...,0.00000,0.00000,0.00000,0.000,97.22544,1,False,False,True,False


,X,Y,distToBuilding,distToCarpark,distToCourtTrack,distToPath,distToRoad,distToWalkway,distToTree,LOTKEY,VegetatedRidge,TempArea,terrain,0,Zone 1,Zone 3
0,21211.76,31449.25,35.13702,40.96888,5.76221,32.17034,22.79395,21.34219,0.00009,MK03-05112L,0,TempArea,24.44757,False,True,False
1,21211.76,31448.25,34.28551,40.84101,6.46347,31.29244,22.37293,20.91852,0.00008,MK03-05112L,0,TempArea,25.59470,False,True,False
2,21211.76,31447.25,33.44222,40.73729,6.01520,30.41453,21.95191,20.49564,0.00007,MK03-05112L,0,TempArea,25.59470,False,True,False
3,21211.76,31446.25,32.60780,40.64762,5.70867,29.53662,21.53089,20.07277,0.00006,MK03-05112L,0,TempArea,25.59470,False,True,False
4,21211.76,31445.25,31.78293,40.55804,5.56732,28.66267,21.10987,19.64989,0.00006,MK03-05112L,0,TempArea,25.59470,False,True,False


# Regression kriging

In [ ]:
allstations=np.array(range(14))
targetstationid=7
otherstationsid=np.delete(allstations,targetstationid).tolist()

In [ ]:
import torch.nn as nn
from torch.autograd import Variable

percentfeatures=['percentroad','percentpath','percentwalkway','percentcourttrack','percentcarpark']

class MyModel(nn.Module):
    def __init__(self, embedding_length, hidden_size,output_size,batch_size):
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        super(MyModel, self).__init__()
        self.lstm = nn.LSTM(embedding_length, hidden_size,batch_first=True)
        self.label = nn.Linear(hidden_size, output_size)
    def forward(self, input):
        h_0 = Variable(torch.zeros(1,self.batch_size, self.hidden_size))
        c_0 = Variable(torch.zeros(1,self.batch_size, self.hidden_size))
        output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
        return self.label(final_hidden_state[-1]) 
class gru(nn.Module):
    def __init__(self, embedding_length, hidden_size,output_size,batch_size):
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        super(gru, self).__init__()
        self.lstm = nn.GRU(embedding_length, hidden_size,batch_first=True)
        self.label = nn.Linear(hidden_size, output_size)
    def forward(self, input):
        h_0 = Variable(torch.zeros(1,self.batch_size, self.hidden_size))
        
        output, final_hidden_state = self.lstm(input, h_0)
        return self.label(final_hidden_state[-1]) 

embedding_length=15
batchsize=16
mymodel=gru(embedding_length=embedding_length,hidden_size=64,output_size=13,batch_size=batchsize)
optimizer = torch.optim.SGD(mymodel.parameters(), lr=0.01, momentum=0.9)

def train_model(model, train_iter, epoch):
    model.train()
    for e in range(epoch):
        loss=0
        optimizer.zero_grad()
        for idx, batch in enumerate(train_iter):
            prediction = model(batch.x)
            loss = loss + torch.norm(prediction-batch.y,"fro")
        loss=loss/(idx+1)
        print("epoch {}: Training Loss normalized Root MSE : {} %".format(e,np.sqrt(loss.detach().numpy())) )
        loss.backward()
        optimizer.step()
        if loss <0.1:
            break
    return loss

class Mydata():
    def __init__(self,x,y):
        self.x=x
        self.y=y

measure='Tem0719'
npdata=np.array(pd.read_csv(dir + measure +'.csv').iloc[:,2:16])
hourdata=np.sum(npdata.reshape([-1,60,14]),axis=1)/60
testhourdata=np.sum(np.array(testdf.iloc[:,2:16]).reshape([-1,60,14]),axis=1)/60
mean = hourdata.mean()
std = hourdata.std()
pdhourdata=pd.DataFrame(hourdata).transpose()
pdhourdata['X']=list(wsdf['X'])
pdhourdata['Y']=list(wsdf['Y'])

    

In [ ]:
measure='Tem0719'
npdata=np.array(pd.read_csv(dir + measure +'.csv').iloc[:,2:16])
hourdata=np.sum(npdata.reshape([-1,60,14]),axis=1)/60
testhourdata=np.sum(np.array(testdf.iloc[:,2:16]).reshape([-1,60,14]),axis=1)/60
mean = hourdata.mean()
std = hourdata.std()

data=[]
testdata=[]
allstations=np.array(range(14))
targetstationid=7
otherstationsid=np.delete(allstations,targetstationid).tolist()
with open('trainednw\\{}lstmmodel_target{}.pkl'.format(measure,targetstationid), 'rb') as f:
        mymodel = pickle.load(f)
if batchsize ==1:
    for i in range(0,hourdata.shape[0]-embedding_length-batchsize,batchsize):
        data.append(Mydata(torch.tensor((np.delete(hourdata[i:i+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32).transpose(0,1),torch.tensor((np.delete(hourdata[i+embedding_length:i+1+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))
    for i in range(0,testhourdata.shape[0]-embedding_length-batchsize,batchsize):
        testdata.append(Mydata(torch.tensor((np.delete(testhourdata[i:i+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32).transpose(0,1),torch.tensor((np.delete(testhourdata[i+embedding_length:i+1+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))

else:
    for i in range(0,hourdata.shape[0]-embedding_length-batchsize,batchsize):
        data.append(Mydata(torch.tensor(np.array([(np.delete(hourdata[i+k:i+k+embedding_length,:],targetstationid,axis=1)-mean)/std for k in range(batchsize)]),dtype=torch.float32).transpose(1,2)
                    ,torch.tensor((np.delete(hourdata[i+embedding_length:i+batchsize+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))
    for i in range(0,testhourdata.shape[0]-embedding_length-batchsize,batchsize):
        testdata.append(Mydata(torch.tensor(np.array([(np.delete(testhourdata[i+k:i+k+embedding_length,:],targetstationid,axis=1)-mean)/std for k in range(batchsize)]),dtype=torch.float32).transpose(1,2)
                    ,torch.tensor((np.delete(testhourdata[i+embedding_length:i+batchsize+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))

griddflist=[]
plotdflist=[]
for i, mth in enumerate(months):
    griddflist.append(griddf.copy())
    plotdflist.append(plotdf.copy())
    print("Date : {}, Time : {}".format(df.iloc[(mth*batchsize+batchindex+embedding_length)*60,0],df.iloc[(mth*batchsize+batchindex+embedding_length)*60,1]))

    x = np.array(list(zip(wsdf.drop(targetstationid).X, wsdf.drop(targetstationid).Y)))
    truevalue = hourdata[mth*batchsize+batchindex+embedding_length,:]
    lastdayvalue = hourdata[mth*batchsize+batchindex+embedding_length-24,:]
    target = mymodel(data[mth].x)[batchindex,:].detach().numpy()*std+mean
    # p_train, p_test, x_train, x_test, target_train, target_test = train_test_split(
    #     p, x, target, test_size=0.3, random_state=42
    # )
    plotdf['plotdata']=truevalue
    print("=" * 40)
    m_ok = OrdinaryKriging(x[:,0],x[:,1],target,variogram_model=k,verbose=False)
    m_rk = RegressionKriging(regression_model=model, n_closest_points=n, variogram_model=k, verbose=False)

    # m_rk.fit(p_train, x_train,target_train)
    # score = m_rk.score(p_test, x_test,target_test)
    # scores_mths.append(score)

    m_rk.fit(p, x, target)
    result = m_rk.predict(target_p, target_x)
    griddflist[i][measure+'_lstmrk_'] = result

NameError: name 'months' is not defined

In [ ]:
error=[]
for k in variogram_models:
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    for mth in range(len(data)):
        for batchindex in range(batchsize):
            truevalue = hourdata[mth*batchsize+batchindex+embedding_length,targetstationid]
            target = mymodel(data[mth].x)[batchindex,:].detach().numpy()*std+mean
            m_rk = RegressionKriging(regression_model=model, n_closest_points=n, variogram_model=k, verbose=False)
            m_rk.fit(p, x, target)
            result = m_rk.predict(target_p, target_x)
            plotdf.iloc[[targetstationid],embedding_length+mth*batchsize+batchindex]=(result-truevalue)

# Model training

In [ ]:
## Select features for model, excluded most of the percent** because they seem quite small and not significant, especially for features with small area like walkways
features = ['terrain','distToBuilding','distToTree','distToWalkway','distToRoad','distToPath','distToCourtTrack','distToCarpark']
p = wsdf[features].drop(targetstationid)
target_p = wsdf.loc[targetstationid,features].to_numpy().reshape(1,-1)
target_x = np.array([(wsdf.loc[targetstationid,'X'],wsdf.loc[targetstationid,'Y'])]).reshape(1,-1)

feature_importance_df = pd.DataFrame(features, columns =['FeatureName'])

## prepare data for training and models to train
data=[]
testdata=[]
if batchsize ==1:
    for i in range(0,hourdata.shape[0]-embedding_length-batchsize,batchsize):
        data.append(Mydata(torch.tensor((np.delete(hourdata[i:i+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32).transpose(0,1),torch.tensor((np.delete(hourdata[i+embedding_length:i+1+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))
    for i in range(0,testhourdata.shape[0]-embedding_length-batchsize,batchsize):
        testdata.append(Mydata(torch.tensor((np.delete(testhourdata[i:i+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32).transpose(0,1),torch.tensor((np.delete(testhourdata[i+embedding_length:i+1+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))

else:
    for i in range(0,hourdata.shape[0]-embedding_length-batchsize,batchsize):
        data.append(Mydata(torch.tensor(np.array([(np.delete(hourdata[i+k:i+k+embedding_length,:],targetstationid,axis=1)-mean)/std for k in range(batchsize)]),dtype=torch.float32).transpose(1,2)
                    ,torch.tensor((np.delete(hourdata[i+embedding_length:i+batchsize+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))
    for i in range(0,testhourdata.shape[0]-embedding_length-batchsize,batchsize):
        testdata.append(Mydata(torch.tensor(np.array([(np.delete(testhourdata[i+k:i+k+embedding_length,:],targetstationid,axis=1)-mean)/std for k in range(batchsize)]),dtype=torch.float32).transpose(1,2)
                    ,torch.tensor((np.delete(testhourdata[i+embedding_length:i+batchsize+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))


methods=['_lstmrk_','_lstmok_','_grurk_','_gruok_','_ne_','_iwec_','_changi_']#'
puredata={}
for method in methods:
    puredata[measure+method]=plotdf.iloc[:,embedding_length:embedding_length+len(data)*batchsize].to_numpy()
### SVR model parameters
C = 0.0005
gamma = 5
kernel = ['linear'] # options: ['linear', 'poly', 'rbf', 'sigmoid']

### RandomForestRegressor parameters
n_estimators=50
random_state=4

### RegressionKrigging parameters
n = 8
variogram_models = ['spherical'] # options: ["linear", "power", "gaussian", "spherical", "exponential"]

#months = ["Feb-19","Mar-19","Apr-19","May-19","Jun-19","Jul-19","Aug-19","Sep-19","Oct-19","Nov-19","Dec-19","Jan-20","Feb-20","Mar-20","Apr-20"]
months = [12,31,5]
#months = [21]
batchindex = 5
### Load base geojson grid to export geojson file
basejson = []
# with open(dir + 'GeoJSON/BaseGrid.geojson', 'r') as file:
#     basejson = json.load(file)
dt=[]
for i, mth in enumerate(months):
    dt.append("Date : {}, Time : {}".format(df.iloc[(mth*batchsize+batchindex+embedding_length)*60,0],df.iloc[(mth*batchsize+batchindex+embedding_length)*60,1]))
    plotdf[measure+'_rk_'+dt[i]]=0
    plotdf[measure+'_ok_'+dt[i]]=0
    plotdf[measure+'_ne_'+dt[i]]=0
for targetstationid in range(14):    
    allstations=np.array(range(14))
    otherstationsid=np.delete(allstations,targetstationid).tolist()
    import pickle
    data=[]
    testdata=[]
    if batchsize ==1:
        for i in range(0,hourdata.shape[0]-embedding_length-batchsize,batchsize):
            data.append(Mydata(torch.tensor((np.delete(hourdata[i:i+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32).transpose(0,1),torch.tensor((np.delete(hourdata[i+embedding_length:i+1+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))
        for i in range(0,testhourdata.shape[0]-embedding_length-batchsize,batchsize):
            testdata.append(Mydata(torch.tensor((np.delete(testhourdata[i:i+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32).transpose(0,1),torch.tensor((np.delete(testhourdata[i+embedding_length:i+1+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))

    else:
        for i in range(0,hourdata.shape[0]-embedding_length-batchsize,batchsize):
            data.append(Mydata(torch.tensor(np.array([(np.delete(hourdata[i+k:i+k+embedding_length,:],targetstationid,axis=1)-mean)/std for k in range(batchsize)]),dtype=torch.float32).transpose(1,2)
                        ,torch.tensor((np.delete(hourdata[i+embedding_length:i+batchsize+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))
        for i in range(0,testhourdata.shape[0]-embedding_length-batchsize,batchsize):
            testdata.append(Mydata(torch.tensor(np.array([(np.delete(testhourdata[i+k:i+k+embedding_length,:],targetstationid,axis=1)-mean)/std for k in range(batchsize)]),dtype=torch.float32).transpose(1,2)
                        ,torch.tensor((np.delete(testhourdata[i+embedding_length:i+batchsize+embedding_length,:],targetstationid,axis=1)-mean)/std,dtype=torch.float32)))

    with open('TEMmodel_target{}.pkl'.format(targetstationid), 'rb') as f:
        mymodel = pickle.load(f)
    #mymodel=MyModel(embedding_length=embedding_length,hidden_size=64,output_size=13,batch_size=batchsize)
    train_model(mymodel, data, epoch=1000)
    with open('TEMmodel_target{}.pkl'.format(targetstationid),'wb') as f:
        pickle.dump(mymodel,f)
    #mymodel=clf2
    

    for k in variogram_models:
        # print(k)
        # model = SVR(C=C, gamma=gamma, kernel=k)
        model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
        # model = LinearRegression(copy_X=True, fit_intercept=False)

        scores_mths = []
        #
        #   continue
        
        for i, mth in enumerate(months):
            
            x = np.array(list(zip(wsdf.drop(targetstationid).X, wsdf.drop(targetstationid).Y)))
            truevalue = hourdata[mth*batchsize+batchindex+embedding_length,:]
            target = mymodel(data[mth].x)[batchindex,:].detach().numpy()*std+mean
            # p_train, p_test, x_train, x_test, target_train, target_test = train_test_split(
            #     p, x, target, test_size=0.3, random_state=42
            # )
            print("=" * 40)
            m_ok = OrdinaryKriging(x[:,0],x[:,1],target,variogram_model=k,verbose=False)
            m_rk = RegressionKriging(regression_model=model, n_closest_points=n, variogram_model=k, verbose=False)

            # m_rk.fit(p_train, x_train,target_train)
            # score = m_rk.score(p_test, x_test,target_test)
            # scores_mths.append(score)

            m_rk.fit(p, x, target)
            result = m_rk.predict(target_p, target_x)
            plotdf.loc[:,measure+'_rk_'+dt[i]].iloc[[targetstationid]] = result[0]-truevalue[targetstationid]
            z,sigma=m_ok.execute('points',target_x[0,0],target_x[0,1])
            plotdf.loc[:,measure+'_ok_'+dt[i]].iloc[[targetstationid]] = z[0]-truevalue[targetstationid]
            plotdf.loc[:,measure+'_ne_'+dt[i]].iloc[[targetstationid]] = truevalue[otherstationsid[np.argmin((wsdf.loc[otherstationsid,'X']-wsdf.loc[targetstationid,'X'])**2+(wsdf.loc[otherstationsid,'Y']-wsdf.loc[targetstationid,'Y'])**2)]]-truevalue[targetstationid]
            
for i, mth in enumerate(months):
    print(dt[i])        
    for method in methods:
        g = sns.scatterplot(x="X", y="Y",
                        hue=measure+method+dt[i],
                        palette="Spectral_r",
                        data=plotdf.iloc[otherstationsid,:],
                        #hue_norm=(-1,1),
                        edgecolor="black")
        plt.title("Temperature prediction error samples, Date : {}, Time : {}".format(df.iloc[(mth*batchsize+batchindex+embedding_length)*60,0],df.iloc[(mth*batchsize+batchindex+embedding_length)*60,1]))
        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
        plt.show()
        # plt.savefig(dir + 'Export/' + measure + mth + '.png', bbox_inches='tight')
        #resultdf.to_csv(dir + 'Export/' + measure + mth + '.csv')
    error=result-truevalue
    display(error)

        ### Add properties to geojson and export
        # Tmaxes = resultdf['Tmax'].tolist()
        # for i in range(len(resultdf.index)):
        #     basejson['features'][i]['properties']['Tmax'] = Tmaxes[i]
        # with open(dir + 'Export/' + measure + mth + '.geojson', 'w') as outfile:
        #     json.dump(basejson, outfile)

    # avg = sum(scores_mths) / len(scores_mths)
    # scores_all.append([k,C,gamma,n] + scores_mths + [avg] )


fig, axes = plt.subplots(clusternum, 1, figsize=(10, 16), sharex=True)
for clusterid,ax in enumerate(axes):
    sns.boxplot(data = puredata[plotdf['cluster']==clusterid].to_numpy().reshape(-1,24), ax=ax) 
    ax.set_ylabel("") 
    ax.set_title("Temperature prediction error of stations in cluster {}".format(clusterid)) 
    if ax != axes[-1]: 
        ax.set_xlabel('')
    else:
        ax.set_xlabel("time in the day")
        

In [ ]:
#save the training results
'''
for targetstationid in range(14):
    mymodel = gru(embedding_length=embedding_length,hidden_size=64,output_size=13,batch_size=batchsize)
    with open('trainednw\\{}grumodel_target{}.pkl'.format(measure,targetstationid),'wb') as f:
        pickle.dump(mymodel,f)
    mymodel = MyModel(embedding_length=embedding_length,hidden_size=64,output_size=13,batch_size=batchsize)
    with open('trainednw\\{}lstmmodel_target{}.pkl'.format(measure,targetstationid),'wb') as f:
        pickle.dump(mymodel,f)
'''